## Preprocessing text for LLM training ¶

In [1]:
import os
import urllib.request
import torch

In [2]:
#File opening
with open("the-verdict.txt", "r", encoding="UTF-8") as f:
    raw_text = f.read()
print("Total number of characters:",len(raw_text))
print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
#Using regular expression library to split text
import re 
text = "Hello, World. This, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'World.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [4]:
#Improving our code to include more special characters
text = "Hello, World. Is this-- a test?"
result = re.split(r'([,.:;?"_!()\']|--|\s)',text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'World', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [5]:
#Applying our tokenizer to entire Edith Wharton's Story
preprocessed = re.split(r'([,.:;?"_!()\']|--|\s)',raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


In [6]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [7]:
#Assigning token IDs
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [12]:
#Creating a vocabulary layer 
vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i>=50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [8]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)   
        return text

In [13]:
#instanstiating a tokenizer object
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," 
       Mrs. Gisburn said with pardonable pride."""
encoded_output = tokenizer.encode(text)
print(encoded_output)


[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [14]:
decoded_output = tokenizer.decode(encoded_output)
print(decoded_output)

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [15]:
#Next step is to add special context tokens that can handle unknown words, why?
text = "Hello! is this a test?"
output = tokenizer.encode(text)

KeyError: 'Hello'

In [16]:
#Therefore the error above shows that the word 'Hello' isnt recognised because it is not in our vocabulary, therefore we must handle it using special context tokens
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [17]:
#updating our tokenizer class 
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}
    def encode(self, text):
        preprocessed = re.split(r'([,.?:;"!_()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [18]:
tokenizerv2 = SimpleTokenizerV2(vocab)
test = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace."
print(tokenizerv2.encode(test))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [19]:
print(tokenizerv2.decode(tokenizerv2.encode(test)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


## Byte pair encoding ¶

Another Nice method is to implement Byte-Pair Encoding which breaks down unknown words into subwords or subwords into characeters and then assings them token ids, BPE algorithm iteratively builds its own vocabulary this way.

In [20]:
#Encoding using BPE
import tiktoken 
tokenizer = tiktoken.get_encoding("gpt2")
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [22]:
#Decoding 
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


## Data Sampling with a Sliding Window ¶
We will go through PyTorch's DataSet and DataLoader classes using a simple example and then implement it for our LLM, but before that we will implement the sliding window technique for generating input output pairs on our own.

In [23]:
#We will now get a feel for how convenient BPE tokenizer is and encode the entire Edith Wharton Story
with open("the-verdict.txt","r", encoding="UTF-8") as f:
    raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [24]:
#now we take a sample for data and try to create input-output pairs using 2 variables
enc_sample = enc_text[50:]

In [25]:
#Now we create a list of input and another list of target ouputs first
context_size = 4 #Determines the number of tokens in input
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"X:{x}")
print(f"y:     {y}")

X:[290, 4920, 2241, 287]
y:     [4920, 2241, 287, 257]


In [26]:
#next word prediction task
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [27]:
#lets repeat the code, but this time we will convert token id to text
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [2]:
#Implementing the dataset class 
import torch 
from torch.utils.data import Dataset

X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
], dtype=torch.float32)

y_test = torch.tensor([0, 1])

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index):   
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        return len(self.labels)    

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

# Quick check
print(len(train_ds))  # 5
print(train_ds[0])    # (tensor([-1.2000,  3.1000]), tensor(0))


5
(tensor([-1.2000,  3.1000]), tensor(0))


In [3]:
#Simple implementation of the dataset 
class customdataset(Dataset):
    def __init__(Self, data, labels):
        self.data = data
        self.labels = labels
    def __len__(self):
        return len(self.data)
    def __get_item__(self, index):
        sample = self.data[index]
        labels = self.labels[index]
        return sample, labels

In [4]:
import torch
from torch.utils.data import Dataset

X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
], dtype=torch.float32)

y_test = torch.tensor([0, 1])

class ToyDataset(Dataset):
    def __init__(self, X,y):
        self.features = X
        self.labels = y
    def __getitem__(self,index):
        sample = self.features[index]
        label = self.labels[index]
        return sample, label
    def __len__(self):
        return len(self.labels)
    
    
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)
    
print(len(train_ds))  
print(train_ds[0]) 

5
(tensor([-1.2000,  3.1000]), tensor(0))


In [5]:
#Imlementing proper dataset loader
import torch 
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)
        
        for i in range (0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return (len(self.input_ids))
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]
        
    

In [6]:
#Creating a dataloader function
def create_dataloader_v1(txt, batch_size = 4, max_length = 256 , stride = 128, shuffle = True, drop_last = True, num_workers = 0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers= num_workers
    )
    
    return dataloader

In [8]:
#Testing our dataloader 
import tiktoken
with open ("the-verdict.txt", "r", encoding="UTF-8") as f:
    raw_text = f.read()
dataloader = create_dataloader_v1(raw_text,  batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print("The first batch is: ", first_batch)

The first batch is:  [tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


## Creating token embeddings ¶

In [9]:
#Creating Token Embedding layer hands on
torch.manual_seed(123)
vocab_size = 6
output_dim = 3
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [10]:
#Practice
import torch
from torch.utils.data import Dataset, DataLoader
import tiktoken
class GPTDatasetV2(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)
        
        for i in range (0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i : i + max_length]
            output_chunk = token_ids[i + 1 : i + max_length + 1]
            
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(output_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]
         

In [11]:
def create_dataloader_v2(txt, batch_size = 4, max_length = 256 , stride = 128, shuffle = True, drop_last = True, num_workers = 0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV2(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers= num_workers
    )
    return dataloader

In [12]:
#testing our dataloader
with open("the-verdict.txt", "r", encoding="UTF-8") as f:
    raw_text = f.read()
dataloader = create_dataloader_v2(raw_text,  batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print("The first batch is: ", first_batch)

The first batch is:  [tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [ ]:
#Creating an embedding layer for our input tokens
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride = max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n",inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [ ]:
#Mapping our inputs to a 256 dimension vector
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [ ]:
#Similaryly we create a positional embedding layer and map a positional embedding tensor to 256 dimensions
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [ ]:
#final embeddings are a sum of positional embeddings and token embeddings
input_embeddings = pos_embeddings + token_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
